In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler, Imputer
from sklearn.preprocessing import PolynomialFeatures
from scipy import stats
from scipy.stats import boxcox
pd.options.mode.chained_assignment = None

In [17]:
#Read in Data
train = pd.read_csv('dataFiles/application_train.csv')
test = pd.read_csv('dataFiles/application_test.csv')
bureau_data = pd.read_csv('dataFiles/bureau.csv')
bureau_balance_data = pd.read_csv('dataFiles/bureau_balance.csv')
prev_app_data = pd.read_csv('dataFiles/previous_application.csv')
pos_cash_balance_data = pd.read_csv('dataFiles/POS_CASH_balance.csv')
installments_data = pd.read_csv('dataFiles/installments_payments.csv')
cc_data = pd.read_csv('dataFiles/credit_card_balance.csv')

In [31]:
train_num = train.select_dtypes(exclude='object')
test_num = test.select_dtypes(exclude='object')

In [32]:
#Log Transformations
train_num[['AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY',\
       'AMT_GOODS_PRICE']] = np.log(train_num[['AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY','AMT_GOODS_PRICE']])

test_num[['AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY',\
       'AMT_GOODS_PRICE']] = np.log(test_num[['AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY','AMT_GOODS_PRICE']])

In [33]:
#Clipping Outliers
train_num['CNT_CHILDREN'] = train_num['CNT_CHILDREN'].apply(lambda x: 5 if x >=5 else x)
train_num['CNT_FAM_MEMBERS'] = train_num['CNT_FAM_MEMBERS'].apply(lambda x: 5 if x >=5 else x)
train_num['AMT_INCOME_TOTAL'] = train_num['AMT_INCOME_TOTAL'].apply(lambda x: 14 if x >=14 else x)
train_num['HOUR_APPR_PROCESS_START'] = train_num['HOUR_APPR_PROCESS_START'].apply(lambda x: 2 if x <= 2 else x)
train_num['COMMONAREA_AVG'] = train_num['COMMONAREA_AVG'].fillna(0).apply(lambda x: 0.3 if x >= 0.3 else x)
train_num['AMT_REQ_CREDIT_BUREAU_YEAR'] = train_num['AMT_REQ_CREDIT_BUREAU_YEAR'].apply(lambda x: 9 if x >= 9 else x).fillna(10)

test_num['CNT_CHILDREN'] = test_num['CNT_CHILDREN'].apply(lambda x: 5 if x >=5 else x)
test_num['CNT_FAM_MEMBERS'] = test_num['CNT_FAM_MEMBERS'].apply(lambda x: 5 if x >=5 else x)
test_num['AMT_INCOME_TOTAL'] = test_num['AMT_INCOME_TOTAL'].apply(lambda x: 14 if x >=14 else x)
test_num['HOUR_APPR_PROCESS_START'] = test_num['HOUR_APPR_PROCESS_START'].apply(lambda x: 2 if x <= 2 else x)
test_num['COMMONAREA_AVG'] = test_num['COMMONAREA_AVG'].fillna(0).apply(lambda x: 0.3 if x >= 0.3 else x)
test_num['AMT_REQ_CREDIT_BUREAU_YEAR'] = test_num['AMT_REQ_CREDIT_BUREAU_YEAR'].apply(lambda x: 9 if x >= 9 else x).fillna(10)

In [34]:
#BINNING THE TWO HUMPS FROM DAYS_EMPLOYED
train_num['DAYS_EMPLOYED_BIN_1'] = train_num['DAYS_EMPLOYED'].apply(lambda x: 1 if x < 150000 else 0)
train_num['DAYS_EMPLOYED_BIN_2'] = train_num['DAYS_EMPLOYED'].apply(lambda x: 1 if x >= 150000 else 0)

#BINNING THE TWO HUMPS FROM DAYS_EMPLOYED
test_num['DAYS_EMPLOYED_BIN_1'] = test_num['DAYS_EMPLOYED'].apply(lambda x: 1 if x < 150000 else 0)
test_num['DAYS_EMPLOYED_BIN_2'] = test_num['DAYS_EMPLOYED'].apply(lambda x: 1 if x >= 150000 else 0)

In [35]:
#ABS & BOX COX Transformation
train_num['DAYS_REGISTRATION'] = np.abs(train_num['DAYS_REGISTRATION'])
train_num['DAYS_REGISTRATION'] = train_num['DAYS_REGISTRATION'].apply(lambda x: 0.01 if x == 0 else x)
train_num['DAYS_REGISTRATION'] = boxcox(train_num['DAYS_REGISTRATION'],0.5)

train_num['DAYS_ID_PUBLISH'] = np.abs(train_num['DAYS_ID_PUBLISH'])
train_num['DAYS_ID_PUBLISH'] = train_num['DAYS_ID_PUBLISH'].apply(lambda x: 0.01 if x == 0 else x)
train_num['DAYS_ID_PUBLISH'] = boxcox(train_num['DAYS_ID_PUBLISH'],0.5)

train_num['ENTRANCES_AVG'] = train_num['ENTRANCES_AVG'].apply(lambda x: np.sqrt(x) if pd.notnull(x) else x)

test_num['DAYS_REGISTRATION'] = np.abs(test_num['DAYS_REGISTRATION'])
test_num['DAYS_REGISTRATION'] = test_num['DAYS_REGISTRATION'].apply(lambda x: 0.01 if x == 0 else x)
test_num['DAYS_REGISTRATION'] = boxcox(test_num['DAYS_REGISTRATION'],0.5)

test_num['DAYS_ID_PUBLISH'] = np.abs(test_num['DAYS_ID_PUBLISH'])
test_num['DAYS_ID_PUBLISH'] = test_num['DAYS_ID_PUBLISH'].apply(lambda x: 0.01 if x == 0 else x)
test_num['DAYS_ID_PUBLISH'] = boxcox(test_num['DAYS_ID_PUBLISH'],0.5)

test_num['ENTRANCES_AVG'] = test_num['ENTRANCES_AVG'].apply(lambda x: np.sqrt(x) if pd.notnull(x) else x)

In [36]:
#4 Bins for Own Car Age
def own_car_age_bins(x):
    if pd.isnull(x):
        return 0
    elif x <= 10:
        return 1
    elif x <= 30:
        return 2
    else:
        return 3
    
train['OWN_CAR_AGE_BINS'] = train['OWN_CAR_AGE'].apply(own_car_age_bins)

test['OWN_CAR_AGE_BINS'] = test['OWN_CAR_AGE'].apply(own_car_age_bins)

In [91]:
#Columns to drop because of sparse features
train_num = train_num.drop(['FLAG_MOBIL','FLAG_CONT_MOBILE','FLAG_DOCUMENT_2','FLAG_DOCUMENT_4',\
                    'FLAG_DOCUMENT_7','FLAG_DOCUMENT_10','FLAG_DOCUMENT_12','FLAG_DOCUMENT_15',\
                    'FLAG_DOCUMENT_17','FLAG_DOCUMENT_19','FLAG_DOCUMENT_20','FLAG_DOCUMENT_21','OWN_CAR_AGE','DAYS_EMPLOYED','DAYS_EMPLOYED_BIN_1'], axis = 1)

test_num = test_num.drop(['FLAG_MOBIL','FLAG_CONT_MOBILE','FLAG_DOCUMENT_2','FLAG_DOCUMENT_4',\
                    'FLAG_DOCUMENT_7','FLAG_DOCUMENT_10','FLAG_DOCUMENT_12','FLAG_DOCUMENT_15',\
                    'FLAG_DOCUMENT_17','FLAG_DOCUMENT_19','FLAG_DOCUMENT_20','FLAG_DOCUMENT_21','OWN_CAR_AGE','DAYS_EMPLOYED','DAYS_EMPLOYED_BIN_1'], axis = 1)

ValueError: labels ['FLAG_MOBIL' 'FLAG_CONT_MOBILE' 'FLAG_DOCUMENT_2' 'FLAG_DOCUMENT_4'
 'FLAG_DOCUMENT_7' 'FLAG_DOCUMENT_10' 'FLAG_DOCUMENT_12'
 'FLAG_DOCUMENT_15' 'FLAG_DOCUMENT_17' 'FLAG_DOCUMENT_19'
 'FLAG_DOCUMENT_20' 'FLAG_DOCUMENT_21' 'OWN_CAR_AGE'] not contained in axis

In [39]:
#FillNA Columns
train_num[['APARTMENTS_AVG','BASEMENTAREA_AVG','YEARS_BEGINEXPLUATATION_AVG','YEARS_BUILD_AVG','ELEVATORS_AVG','ENTRANCES_AVG',\
       'AMT_REQ_CREDIT_BUREAU_HOUR','AMT_REQ_CREDIT_BUREAU_DAY','AMT_REQ_CREDIT_BUREAU_WEEK','AMT_REQ_CREDIT_BUREAU_MON',\
       'AMT_REQ_CREDIT_BUREAU_QRT','OBS_30_CNT_SOCIAL_CIRCLE','DEF_30_CNT_SOCIAL_CIRCLE','OBS_60_CNT_SOCIAL_CIRCLE',\
       'DEF_60_CNT_SOCIAL_CIRCLE','EXT_SOURCE_1',\
       'EXT_SOURCE_2','EXT_SOURCE_3']] = train_num[['APARTMENTS_AVG','BASEMENTAREA_AVG','YEARS_BEGINEXPLUATATION_AVG','YEARS_BUILD_AVG',\
                                             'ELEVATORS_AVG','ENTRANCES_AVG','AMT_REQ_CREDIT_BUREAU_HOUR','AMT_REQ_CREDIT_BUREAU_DAY',\
                                             'AMT_REQ_CREDIT_BUREAU_WEEK','AMT_REQ_CREDIT_BUREAU_MON','AMT_REQ_CREDIT_BUREAU_QRT',\
                                              'OBS_30_CNT_SOCIAL_CIRCLE','DEF_30_CNT_SOCIAL_CIRCLE','OBS_60_CNT_SOCIAL_CIRCLE','DEF_60_CNT_SOCIAL_CIRCLE',\
                                              'EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']].fillna(0)

test_num[['APARTMENTS_AVG','BASEMENTAREA_AVG','YEARS_BEGINEXPLUATATION_AVG','YEARS_BUILD_AVG','ELEVATORS_AVG','ENTRANCES_AVG','AMT_REQ_CREDIT_BUREAU_HOUR',\
      'AMT_REQ_CREDIT_BUREAU_DAY','AMT_REQ_CREDIT_BUREAU_WEEK','AMT_REQ_CREDIT_BUREAU_MON','AMT_REQ_CREDIT_BUREAU_QRT',\
      'OBS_30_CNT_SOCIAL_CIRCLE','DEF_30_CNT_SOCIAL_CIRCLE','OBS_60_CNT_SOCIAL_CIRCLE','DEF_60_CNT_SOCIAL_CIRCLE','EXT_SOURCE_1',\
       'EXT_SOURCE_2','EXT_SOURCE_3']] = test[['APARTMENTS_AVG','BASEMENTAREA_AVG','YEARS_BEGINEXPLUATATION_AVG',\
                                  'YEARS_BUILD_AVG','ELEVATORS_AVG','ENTRANCES_AVG','AMT_REQ_CREDIT_BUREAU_HOUR',\
                                              'AMT_REQ_CREDIT_BUREAU_DAY','AMT_REQ_CREDIT_BUREAU_WEEK',\
                                              'AMT_REQ_CREDIT_BUREAU_MON','AMT_REQ_CREDIT_BUREAU_QRT',\
                                             'OBS_30_CNT_SOCIAL_CIRCLE','DEF_30_CNT_SOCIAL_CIRCLE','OBS_60_CNT_SOCIAL_CIRCLE','DEF_60_CNT_SOCIAL_CIRCLE',\
                                             'EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']].fillna(0)

In [100]:
train_num = train_num.fillna(0)

test_num = test_num.fillna(0)

In [103]:
#Threshold for NAs
train_num_subset = train_num.dropna(thresh=len(train_num) - 0, axis = 1).drop(['TARGET','SK_ID_CURR'], axis = 1)

In [104]:
test_num_subset = test_num[train_num_subset.columns.tolist()]

In [105]:
scaler = StandardScaler()

In [106]:
scaler.fit(train_num_subset)
train_num_scaled = scaler.transform(train_num_subset)
test_num_scaled = scaler.transform(test_num_subset)

In [107]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV

In [108]:
xgb_model = XGBClassifier()
# gbm_param_grid = {
#     'subsample': np.arange(.05, 1, .05),
#     'max_depth': np.arange(3, 20, 1),
#     'colsample_bytree': np.arange(.1,1.05, .05)
# }

# randomized_neg_mse = RandomizedSearchCV(estimator = xgb_model,
#                                        param_distributions = gbm_param_grid, n_iter = 10, 
#                                        scoring = 'neg_mean_squared_error', cv = 4)

# randomized_neg_mse.fit(train_num_scaled, train.TARGET)

In [109]:
xgb_model.fit(train_num_scaled, train.TARGET)

XGBClassifier(base_score=0.5, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, n_estimators=100,
       nthread=-1, objective='binary:logistic', seed=0, silent=True,
       subsample=1)

In [110]:
preds = pd.DataFrame(xgb_model.predict_proba(test_num_scaled))

In [111]:
submission_xgb = pd.concat([test.SK_ID_CURR, preds], axis=1).drop(0, axis = 1)
submission_xgb.columns = ['SK_ID_CURR', 'Target']

In [112]:
submission_xgb.to_csv('submission_xgb_transformed.csv', index=False)

In [115]:
from imblearn.over_sampling import SMOTE

In [118]:
sm = SMOTE(random_state=42)

X_res, y_res = sm.fit_sample(train_num_scaled, train.TARGET)

In [122]:
from collections import Counter

In [123]:
print('Resampled dataset shape {}'.format(Counter(y_res)))

Resampled dataset shape Counter({0: 282686, 1: 282686})


In [ ]:
train_dummies = pd.get_dummies(train[['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',\
'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'FONDKAPREMONT_MODE','HOUSETYPE_MODE','WALLSMATERIAL_MODE','EMERGENCYSTATE_MODE']]).fillna(0)

test_dummies = pd.get_dummies(test[['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',\
'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'FONDKAPREMONT_MODE','HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE']]).fillna(0)

In [ ]:
train_dummies = train_dummies.drop(['CODE_GENDER_M','CODE_GENDER_XNA','FLAG_OWN_CAR_N','FLAG_OWN_REALTY_N',\
                    'NAME_TYPE_SUITE_Children','NAME_TYPE_SUITE_Group of people','NAME_TYPE_SUITE_Other_A',\
                    'NAME_TYPE_SUITE_Other_B','NAME_INCOME_TYPE_Businessman','NAME_INCOME_TYPE_Maternity leave',\
                   'NAME_INCOME_TYPE_Student','NAME_INCOME_TYPE_Unemployed','NAME_EDUCATION_TYPE_Academic degree',\
                   'NAME_EDUCATION_TYPE_Incomplete higher','NAME_EDUCATION_TYPE_Lower secondary',\
                   'NAME_FAMILY_STATUS_Unknown','NAME_HOUSING_TYPE_Co-op apartment','NAME_HOUSING_TYPE_Municipal apartment',\
                   'NAME_HOUSING_TYPE_Office apartment','NAME_HOUSING_TYPE_Rented apartment','NAME_HOUSING_TYPE_With parents',\
                   'FONDKAPREMONT_MODE_not specified','FONDKAPREMONT_MODE_org spec account','HOUSETYPE_MODE_specific housing',\
                   'HOUSETYPE_MODE_terraced house','WALLSMATERIAL_MODE_Block','WALLSMATERIAL_MODE_Mixed','WALLSMATERIAL_MODE_Monolithic',\
                   'WALLSMATERIAL_MODE_Others','WALLSMATERIAL_MODE_Wooden','EMERGENCYSTATE_MODE_Yes'], axis = 1)

In [ ]:
test_dummies = test_dummies.drop(['CODE_GENDER_M','FLAG_OWN_CAR_N','FLAG_OWN_REALTY_N',\
                    'NAME_TYPE_SUITE_Children','NAME_TYPE_SUITE_Group of people','NAME_TYPE_SUITE_Other_A',\
                    'NAME_TYPE_SUITE_Other_B','NAME_INCOME_TYPE_Businessman',\
                   'NAME_INCOME_TYPE_Student','NAME_INCOME_TYPE_Unemployed','NAME_EDUCATION_TYPE_Academic degree',\
                   'NAME_EDUCATION_TYPE_Incomplete higher','NAME_EDUCATION_TYPE_Lower secondary',\
                   'NAME_HOUSING_TYPE_Co-op apartment','NAME_HOUSING_TYPE_Municipal apartment',\
                   'NAME_HOUSING_TYPE_Office apartment','NAME_HOUSING_TYPE_Rented apartment','NAME_HOUSING_TYPE_With parents',\
                   'FONDKAPREMONT_MODE_not specified','FONDKAPREMONT_MODE_org spec account','HOUSETYPE_MODE_specific housing',\
                   'HOUSETYPE_MODE_terraced house','WALLSMATERIAL_MODE_Block','WALLSMATERIAL_MODE_Mixed','WALLSMATERIAL_MODE_Monolithic',\
                   'WALLSMATERIAL_MODE_Others','WALLSMATERIAL_MODE_Wooden','EMERGENCYSTATE_MODE_Yes'], axis = 1)

In [ ]:
train.info(verbose=True, null_counts=True)

In [ ]:
#fields to impute
'AMT_ANNUITY','AMT_GOODS_PRICE','CNT_FAM_MEMBERS',

In [ ]:
imputer = Imputer()
imputer.fit(train_subset)
train_merged_imputed = pd.DataFrame(imputer.transform(train_subset), columns = train_subset.columns)

In [ ]:
train_merged_subset = train_merged.dropna(thresh=len(train) - 0, axis = 1)

In [ ]:
train.select_dtypes(exclude='object').info()

In [ ]:
#### Bureau Balance ####


In [ ]:
bureau_balance_data_grouped = pd.get_dummies(bureau_balance_data).groupby('SK_ID_BUREAU', as_index=False).agg({'STATUS_1':'count','MONTHS_BALANCE':min,\
                                                                                 'STATUS_C':sum,'STATUS_0':sum,'STATUS_X':sum})

In [ ]:
bureau_balance_data_grouped = bureau_balance_data_grouped.rename(columns={'STATUS_1':'BALANCE_COUNT'})

In [ ]:
bureau_balance_data_grouped['STATUS_X_RATIO'] = bureau_balance_data_grouped['STATUS_X'] / bureau_balance_data_grouped['BALANCE_COUNT'].astype(float)
bureau_balance_data_grouped['STATUS_C_RATIO'] = bureau_balance_data_grouped['STATUS_C'] / bureau_balance_data_grouped['BALANCE_COUNT'].astype(float)
bureau_balance_data_grouped['STATUS_0_RATIO'] = bureau_balance_data_grouped['STATUS_0'] / bureau_balance_data_grouped['BALANCE_COUNT'].astype(float)

In [ ]:
bureau_data = bureau_data.merge(bureau_balance_data_grouped, how = 'left')

In [ ]:
bureau_data[['MONTHS_BALANCE', 'STATUS_X', 'STATUS_C', 'BALANCE_COUNT', 'STATUS_0', 'STATUS_X_RATIO','STATUS_C_RATIO', 'STATUS_0_RATIO']] = bureau_data[['MONTHS_BALANCE', 'STATUS_X', 'STATUS_C', 'BALANCE_COUNT', 'STATUS_0', 'STATUS_X_RATIO','STATUS_C_RATIO', 'STATUS_0_RATIO']].fillna(0)

In [ ]:
##### Bureau Data #####
bureau_data_grouped = bureau_data.select_dtypes(exclude='object').drop('SK_ID_BUREAU', axis = 1).groupby('SK_ID_CURR').sum()
#bureau_data_grouped.columns = ['_'.join(col) if col != ('SK_ID_CURR', '') else col[0] for col in bureau_data_grouped.columns]
bureau_data_grouped = bureau_data_grouped.reset_index()

In [ ]:
#Past Loan Count
loan_count = bureau_data[['SK_ID_CURR','SK_ID_BUREAU']].groupby('SK_ID_CURR', \
                                                   as_index=False)['SK_ID_BUREAU'].count().rename(columns = {'SK_ID_BUREAU':'LOAN_COUNT'})

bureau_data_grouped = bureau_data_grouped.merge(loan_count, how = 'left')

In [ ]:
#Unique Loan Types
unique_loan_count = bureau_data[['SK_ID_CURR','CREDIT_TYPE']].groupby('SK_ID_CURR',\
                                                                      as_index=False).agg({'CREDIT_TYPE':'nunique'}).rename(columns={'CREDIT_TYPE':'UNIQUE_CREDIT_TYPES'})

bureau_data_grouped = bureau_data_grouped.merge(unique_loan_count, how = 'left')

In [ ]:
#Total Active Loans
bureau_data['CREDIT_ACTIVE_BINARY'] = bureau_data['CREDIT_ACTIVE'].apply(lambda x: 1 if x == 'Active' else 0)

active_loan_count = bureau_data[['SK_ID_CURR','CREDIT_ACTIVE_BINARY']].groupby('SK_ID_CURR', \
                                                   as_index=False)['CREDIT_ACTIVE_BINARY'].sum().rename(columns = {'CREDIT_ACTIVE_BINARY':'ACTIVE_LOANS'})

bureau_data_grouped = bureau_data_grouped.merge(active_loan_count, how = 'left')

In [ ]:
#Days Between Successive Past Applications
grp = bureau_data[['SK_ID_CURR', 'SK_ID_BUREAU', 'DAYS_CREDIT']].groupby(by = ['SK_ID_CURR'])
grp1 = grp.apply(lambda x: x.sort_values(['DAYS_CREDIT'], ascending = False)).reset_index(drop = True)

grp1['DAYS_CREDIT1'] = grp1['DAYS_CREDIT']*-1
grp1['DAYS_DIFF'] = grp1.groupby(by = ['SK_ID_CURR'])['DAYS_CREDIT1'].diff()
grp1['DAYS_DIFF'] = grp1['DAYS_DIFF'].fillna(0).astype('uint32')
del grp1['DAYS_CREDIT1'], grp1['DAYS_CREDIT']

In [ ]:
past_app_days = grp1.groupby('SK_ID_CURR', as_index=False)['DAYS_DIFF'].mean()

In [ ]:
bureau_data_grouped = bureau_data_grouped.merge(past_app_days, how = 'left')

In [ ]:
# Days Credit Expires
bureau_data['CREDIT_ENDDATE_BINARY'] = bureau_data['DAYS_CREDIT_ENDDATE'].apply(lambda x: 0 if x < 0 else 1) 

In [ ]:
B1 = bureau_data.loc[bureau_data['CREDIT_ENDDATE_BINARY'] == 1]

In [ ]:
grp = B1[['SK_ID_CURR', 'SK_ID_BUREAU', 'DAYS_CREDIT_ENDDATE']].groupby(by = ['SK_ID_CURR'])
# Sort the values of CREDIT_ENDDATE for each customer ID 
grp1 = grp.apply(lambda x: x.sort_values(['DAYS_CREDIT_ENDDATE'], ascending = True)).reset_index(drop = True)
del grp

In [ ]:
grp1['DAYS_ENDDATE_DIFF'] = grp1.groupby(by = ['SK_ID_CURR'])['DAYS_CREDIT_ENDDATE'].diff()
grp1['DAYS_ENDDATE_DIFF'] = grp1['DAYS_ENDDATE_DIFF'].fillna(0).astype('uint32')
del grp1['DAYS_CREDIT_ENDDATE']

In [ ]:
credit_expires_days = grp1.groupby('SK_ID_CURR', as_index = False)['DAYS_ENDDATE_DIFF'].mean()

bureau_data_grouped = bureau_data_grouped.merge(credit_expires_days, how = 'left')

In [ ]:
# % Active Loans
bureau_data_grouped['ACTIVE_LOAN_PERC'] = bureau_data_grouped['ACTIVE_LOANS'] / bureau_data_grouped['LOAN_COUNT'].astype(float)

In [ ]:
##### Credit Card Data

In [ ]:
cc_data_one_hot = pd.concat([cc_data['SK_ID_PREV'], \
                            pd.get_dummies(cc_data.select_dtypes(include=['object']), drop_first = True)],\
                            axis = 1)

In [ ]:
cc_data_one_hot = cc_data_one_hot.drop(['NAME_CONTRACT_STATUS_Approved','NAME_CONTRACT_STATUS_Demand','NAME_CONTRACT_STATUS_Demand',\
                      'NAME_CONTRACT_STATUS_Refused','NAME_CONTRACT_STATUS_Sent proposal','NAME_CONTRACT_STATUS_Signed'], axis = 1)

In [ ]:
cc_data_one_hot_grouped = cc_data_one_hot.groupby('SK_ID_PREV', as_index=False).sum()

In [ ]:
cc_data_numeric_grouped = cc_data.select_dtypes(exclude=['object']).groupby('SK_ID_PREV', as_index = False).agg(['count', sum, 'mean', min, max])

In [ ]:
cc_data_numeric_grouped.columns = ['_CC_'.join(col) if col != ('SK_ID_PREV', '') else col[0] for col in cc_data_numeric_grouped.columns]

In [ ]:
filtered_cols = filter(lambda x: x[-5:] != 'count' and x[0:10] != 'SK_ID_CURR', cc_data_numeric_grouped.columns.tolist())
filtered_cols.insert(0, 'MONTHS_BALANCE_CC_count')

In [ ]:
cc_data_numeric_grouped_filtered = cc_data_numeric_grouped[filtered_cols].reset_index()

In [ ]:
cc_data_grouped = cc_data_numeric_grouped_filtered.merge(cc_data_one_hot_grouped, how = 'left')

In [ ]:
prev_app_data_dummies = pd.get_dummies(prev_app_data[['NAME_CONTRACT_TYPE','FLAG_LAST_APPL_PER_CONTRACT','NAME_CONTRACT_STATUS',\
                                       'NAME_PAYMENT_TYPE','CODE_REJECT_REASON','NAME_TYPE_SUITE','NAME_CLIENT_TYPE',\
                                       'NAME_PORTFOLIO','CHANNEL_TYPE','NAME_YIELD_GROUP','PRODUCT_COMBINATION']]).fillna(0)

In [ ]:
prev_app_data_dummies = prev_app_data_dummies[['PRODUCT_COMBINATION_Cash X-Sell: middle','NAME_PORTFOLIO_Cards','CODE_REJECT_REASON_HC','NAME_CONTRACT_TYPE_Revolving loans','CHANNEL_TYPE_Stone','NAME_TYPE_SUITE_Family','PRODUCT_COMBINATION_POS mobile with interest','PRODUCT_COMBINATION_POS household with interest','PRODUCT_COMBINATION_Cash','NAME_CONTRACT_STATUS_Refused','NAME_CLIENT_TYPE_New','NAME_CONTRACT_STATUS_Canceled','NAME_YIELD_GROUP_low_normal','NAME_YIELD_GROUP_high','NAME_PORTFOLIO_XNA','NAME_YIELD_GROUP_middle','NAME_PORTFOLIO_Cash','CHANNEL_TYPE_Country-wide','NAME_TYPE_SUITE_Unaccompanied','NAME_YIELD_GROUP_XNA','NAME_PAYMENT_TYPE_XNA','NAME_PORTFOLIO_POS','CHANNEL_TYPE_Credit and cash offices','NAME_CONTRACT_TYPE_Consumer loans','NAME_CONTRACT_TYPE_Cash loans','NAME_PAYMENT_TYPE_Cash through the bank','NAME_CONTRACT_STATUS_Approved','NAME_CLIENT_TYPE_Repeater','CODE_REJECT_REASON_XAP','FLAG_LAST_APPL_PER_CONTRACT_Y']]

In [ ]:
prev_app_data = pd.concat([prev_app_data.select_dtypes(exclude=['object']), prev_app_data_dummies], axis = 1)

In [ ]:
prev_app_data_merged = prev_app_data.merge(cc_data_grouped, how = 'left', left_on = 'SK_ID_PREV', right_on = 'SK_ID_PREV')

In [ ]:
installments_data_grouped = installments_data.groupby('SK_ID_PREV', as_index = False).agg(['count', sum, 'mean', min, max])

In [ ]:
installments_data_grouped.columns = ['_INST_'.join(col) if col != ('SK_ID_PREV', '') else col[0] for col in installments_data_grouped.columns]

In [ ]:
installments_data_grouped = installments_data_grouped.reset_index()

In [ ]:
filtered_cols = filter(lambda x: x[-5:] != 'count' and x[0:10] != 'SK_ID_CURR', installments_data_grouped.columns.tolist())
filtered_cols.insert(0, 'SK_ID_CURR_INST_count')

In [ ]:
installments_data_grouped = installments_data_grouped[filtered_cols]

In [ ]:
prev_app_data_merged = prev_app_data_merged.merge(installments_data_grouped, how = 'left', left_on = 'SK_ID_PREV', right_on = 'SK_ID_PREV')

In [ ]:
pos_cash_balance_data = pd.get_dummies(pos_cash_balance_data)

In [ ]:
pos_cash_balance_data = pos_cash_balance_data.drop(['NAME_CONTRACT_STATUS_Amortized debt',\
                                                    'NAME_CONTRACT_STATUS_Approved','NAME_CONTRACT_STATUS_Canceled','NAME_CONTRACT_STATUS_Demand',\
                                                    'NAME_CONTRACT_STATUS_Returned to the store','NAME_CONTRACT_STATUS_XNA'], axis = 1)

In [ ]:
pos_cash_balance_data_grouped = pos_cash_balance_data.groupby('SK_ID_PREV', as_index=False).agg(['count', sum, 'mean', min, max])

In [ ]:
pos_cash_balance_data_grouped.columns = ['_POS_'.join(col) if col != ('SK_ID_PREV', '') else col[0] for col in pos_cash_balance_data_grouped.columns]

In [ ]:
pos_cash_balance_data_grouped = pos_cash_balance_data_grouped.reset_index()

In [ ]:
filtered_cols = filter(lambda x: x[-5:] != 'count' and x[0:10] != 'SK_ID_CURR', pos_cash_balance_data_grouped.columns.tolist())
filtered_cols.insert(0, 'SK_ID_CURR_POS_count')

In [ ]:
pos_cash_balance_data_grouped = pos_cash_balance_data_grouped[filtered_cols]

In [ ]:
prev_app_data_merged = prev_app_data_merged.merge(pos_cash_balance_data_grouped, how = 'left', left_on = 'SK_ID_PREV', right_on = 'SK_ID_PREV')

In [ ]:
filtered_cols = filter(lambda x: x[-5:] != 'count' and x[0:10] != 'SK_ID_CURR', bureau_data_grouped.columns.tolist())
filtered_cols.insert(0, 'SK_ID_BUREAU_count')

In [ ]:
train_one_hot = pd.concat([train.select_dtypes(exclude=['object']), train_dummies], axis = 1)

test_one_hot = pd.concat([test.select_dtypes(exclude=['object']), test_dummies], axis = 1)

In [ ]:
train_merged = train_one_hot.merge(bureau_data_grouped, how = 'left', left_on = 'SK_ID_CURR', right_on = 'SK_ID_CURR')

test_merged = test_one_hot.merge(bureau_data_grouped, how = 'left', left_on = 'SK_ID_CURR', right_on = 'SK_ID_CURR')

In [ ]:
prev_app_data_subset = prev_app_data_merged[prev_app_data_merged.columns.tolist()].drop('SK_ID_PREV', axis = 1)

In [ ]:
prev_app_data_grouped = prev_app_data_subset.groupby('SK_ID_CURR', as_index=False).sum()

In [ ]:
#prev_app_data_grouped.columns = ['_'.join(col) if col != ('SK_ID_CURR', '') else col[0] for col in prev_app_data_grouped.columns]
#prev_app_data_grouped = prev_app_data_grouped.reset_index()

In [ ]:
train_merged = train_merged.merge(prev_app_data_grouped, how = 'left', left_on = 'SK_ID_CURR', right_on = 'SK_ID_CURR')

test_merged = test_merged.merge(prev_app_data_grouped, how = 'left', left_on = 'SK_ID_CURR', right_on = 'SK_ID_CURR')

In [ ]:
own_car_median = train_merged.OWN_CAR_AGE.median()

In [ ]:
def own_car_missing(x):
    if x['FLAG_OWN_CAR'] == 'N':
        return 28
    elif x['FLAG_OWN_CAR'] == 'Y' and pd.isnull(x['OWN_CAR_AGE']):
        return own_car_median
    else:
        return x['OWN_CAR_AGE']

In [ ]:
train_merged_subset = train_merged.dropna(thresh=len(train_merged) - 200000000, axis = 1)

In [ ]:
train_merged_subset.head()

In [ ]:
train_merged_subset['AMT_CREDIT_y'].hist()

In [ ]:
train_merged_subset.info(verbose=True, null_counts = True)

In [ ]:
column_corr_subset = train_merged_subset.columns.tolist()

In [ ]:
len(column_corr_subset)

In [ ]:
#col_corr = train_merged_subset.corr()['TARGET'].sort_values()

In [ ]:
#column_corr_subset = col_corr[(col_corr >= 0.03) | (col_corr < -0.035)].index.values.tolist()

In [ ]:
column_corr_subset.remove('TARGET')

In [ ]:
#column_corr_subset.remove('CODE_GENDER_XNA')
#column_corr_subset.remove('NAME_FAMILY_STATUS_Unknown')
#column_corr_subset.remove('NAME_INCOME_TYPE_Maternity leave')

In [ ]:
train_subset = train_merged_subset[column_corr_subset]

test_subset = test_merged[column_corr_subset]

In [ ]:
imputer = Imputer()
imputer.fit(train_subset)
train_merged_imputed = pd.DataFrame(imputer.transform(train_subset), columns = train_subset.columns)
test_merged_imputed = pd.DataFrame(imputer.transform(test_subset), columns = train_subset.columns)

In [ ]:
train_merged_imputed['DAYS_EMPLOYED_^2'] = train_merged_imputed['DAYS_EMPLOYED'] ** 2
#train_merged_imputed['AMT_GOODS_PRICE_^2'] = train_merged_imputed['AMT_GOODS_PRICE'] ** 2
train_merged_imputed['DAYS_CREDIT^2'] = train_merged_imputed['DAYS_CREDIT'] ** 2
#train_merged_imputed['DAYS_CREDIT_median^2'] = train_merged_imputed['DAYS_CREDIT_median'] ** 2
train_merged_imputed['DAYS_BIRTH_^2'] = train_merged_imputed['DAYS_BIRTH'] ** 2
train_merged_imputed['REGION_RATING_CLIENT_W_CITY_^2'] = train_merged_imputed['REGION_RATING_CLIENT_W_CITY'] ** 2
train_merged_imputed['REGION_RATING_CLIENT_^2'] = train_merged_imputed['REGION_RATING_CLIENT'] ** 2
train_merged_imputed['NAME_INCOME_TYPE_Working_^2'] = train_merged_imputed['NAME_INCOME_TYPE_Working'] ** 2
train_merged_imputed['DAYS_LAST_PHONE_CHANGE_^2'] = train_merged_imputed['DAYS_LAST_PHONE_CHANGE'] ** 2
train_merged_imputed['EXT_SOURCE_1_^2'] = train_merged_imputed['EXT_SOURCE_1'] ** 2
train_merged_imputed['EXT_SOURCE_2_^2'] = train_merged_imputed['EXT_SOURCE_2'] ** 2
train_merged_imputed['EXT_SOURCE_3_^2'] = train_merged_imputed['EXT_SOURCE_3'] ** 2
train_merged_imputed['NAME_EDUCATION_TYPE_Higher education_^2'] = train_merged_imputed['NAME_EDUCATION_TYPE_Higher education'] ** 2
train_merged_imputed['CODE_GENDER_F_^2']= train_merged_imputed['CODE_GENDER_F'] ** 2

train_merged_imputed['DAYS_EMPLOYED_^3'] = train_merged_imputed['DAYS_EMPLOYED'] ** 3
#train_merged_imputed['AMT_GOODS_PRICE_^3'] = train_merged_imputed['AMT_GOODS_PRICE'] ** 3
train_merged_imputed['DAYS_CREDIT^3'] = train_merged_imputed['DAYS_CREDIT'] ** 3
#train_merged_imputed['DAYS_CREDIT_median^3'] = train_merged_imputed['DAYS_CREDIT_median'] ** 3
train_merged_imputed['DAYS_BIRTH_^3'] = train_merged_imputed['DAYS_BIRTH'] ** 3
train_merged_imputed['REGION_RATING_CLIENT_W_CITY_^3'] = train_merged_imputed['REGION_RATING_CLIENT_W_CITY'] ** 3
train_merged_imputed['REGION_RATING_CLIENT_^3'] = train_merged_imputed['REGION_RATING_CLIENT'] ** 3
train_merged_imputed['NAME_INCOME_TYPE_Working_^3'] = train_merged_imputed['NAME_INCOME_TYPE_Working'] ** 3
train_merged_imputed['DAYS_LAST_PHONE_CHANGE_^3'] = train_merged_imputed['DAYS_LAST_PHONE_CHANGE'] ** 3
train_merged_imputed['EXT_SOURCE_1_^3'] = train_merged_imputed['EXT_SOURCE_1'] ** 3
train_merged_imputed['EXT_SOURCE_2_^3'] = train_merged_imputed['EXT_SOURCE_2'] ** 3
train_merged_imputed['EXT_SOURCE_3_^3'] = train_merged_imputed['EXT_SOURCE_3'] ** 3
train_merged_imputed['NAME_EDUCATION_TYPE_Higher education_^3'] = train_merged_imputed['NAME_EDUCATION_TYPE_Higher education'] ** 3
train_merged_imputed['CODE_GENDER_F_^3']= train_merged_imputed['CODE_GENDER_F'] ** 3

test_merged_imputed['DAYS_EMPLOYED_^2'] = test_merged_imputed['DAYS_EMPLOYED'] ** 2
#test_merged_imputed['AMT_GOODS_PRICE_^2'] = test_merged_imputed['AMT_GOODS_PRICE'] ** 2
test_merged_imputed['DAYS_CREDIT_^2'] = test_merged_imputed['DAYS_CREDIT'] ** 2
#test_merged_imputed['DAYS_CREDIT_median^2'] = test_merged_imputed['DAYS_CREDIT_median'] ** 2
test_merged_imputed['DAYS_BIRTH_^2'] = test_merged_imputed['DAYS_BIRTH'] ** 2
test_merged_imputed['REGION_RATING_CLIENT_W_CITY_^2'] = test_merged_imputed['REGION_RATING_CLIENT_W_CITY'] ** 2
test_merged_imputed['REGION_RATING_CLIENT_^2'] = test_merged_imputed['REGION_RATING_CLIENT'] ** 2
test_merged_imputed['NAME_INCOME_TYPE_Working_^2'] = test_merged_imputed['NAME_INCOME_TYPE_Working'] ** 2
test_merged_imputed['DAYS_LAST_PHONE_CHANGE_^2'] = test_merged_imputed['DAYS_LAST_PHONE_CHANGE'] ** 2
test_merged_imputed['EXT_SOURCE_1_^2'] = test_merged_imputed['EXT_SOURCE_1'] ** 2
test_merged_imputed['EXT_SOURCE_2_^2'] = test_merged_imputed['EXT_SOURCE_2'] ** 2
test_merged_imputed['EXT_SOURCE_3_^2'] = test_merged_imputed['EXT_SOURCE_3'] ** 2
test_merged_imputed['NAME_EDUCATION_TYPE_Higher education_^2'] = test_merged_imputed['NAME_EDUCATION_TYPE_Higher education'] ** 2
test_merged_imputed['CODE_GENDER_F_^2']= test_merged_imputed['CODE_GENDER_F'] ** 2

test_merged_imputed['DAYS_EMPLOYED_^3'] = test_merged_imputed['DAYS_EMPLOYED'] ** 3
#test_merged_imputed['AMT_GOODS_PRICE_^3'] = test_merged_imputed['AMT_GOODS_PRICE'] ** 3
test_merged_imputed['DAYS_CREDIT^3'] = test_merged_imputed['DAYS_CREDIT'] ** 3
#test_merged_imputed['DAYS_CREDIT_median^3'] = test_merged_imputed['DAYS_CREDIT_median'] ** 3
test_merged_imputed['DAYS_BIRTH_^3'] = test_merged_imputed['DAYS_BIRTH'] ** 3
test_merged_imputed['REGION_RATING_CLIENT_W_CITY_^3'] = test_merged_imputed['REGION_RATING_CLIENT_W_CITY'] ** 3
test_merged_imputed['REGION_RATING_CLIENT_^3'] = test_merged_imputed['REGION_RATING_CLIENT'] ** 3
test_merged_imputed['NAME_INCOME_TYPE_Working_^3'] = test_merged_imputed['NAME_INCOME_TYPE_Working'] ** 3
test_merged_imputed['DAYS_LAST_PHONE_CHANGE_^3'] = test_merged_imputed['DAYS_LAST_PHONE_CHANGE'] ** 3
test_merged_imputed['EXT_SOURCE_1_^3'] = test_merged_imputed['EXT_SOURCE_1'] ** 3
test_merged_imputed['EXT_SOURCE_2_^3'] = test_merged_imputed['EXT_SOURCE_2'] ** 3
test_merged_imputed['EXT_SOURCE_3_^3'] = test_merged_imputed['EXT_SOURCE_3'] ** 3
test_merged_imputed['NAME_EDUCATION_TYPE_Higher education_^3'] = test_merged_imputed['NAME_EDUCATION_TYPE_Higher education'] ** 3
test_merged_imputed['CODE_GENDER_F_^3']= test_merged_imputed['CODE_GENDER_F'] ** 3

In [ ]:
poly_transformer = PolynomialFeatures(degree = 1)
poly_transformer.fit(train_merged_imputed)
train_poly_features = poly_transformer.transform(train_merged_imputed)

In [ ]:
train_subset_poly = pd.DataFrame(train_poly_features, columns = poly_transformer.get_feature_names(
    input_features = train_merged_imputed.columns.tolist()
))

In [ ]:
test_poly_features = poly_transformer.transform(test_merged_imputed)

In [ ]:
test_subset_poly = pd.DataFrame(test_poly_features, columns = poly_transformer.get_feature_names(input_features = test_merged_imputed.columns.tolist()))

In [ ]:
scaler = StandardScaler()

In [ ]:
scaler.fit(train_subset_poly)
train_scaled = scaler.transform(train_subset_poly)
test_scaled = scaler.transform(test_subset_poly)

In [ ]:
from catboost import CatBoostClassifier
cat_model = CatBoostClassifier(iterations = 1000, random_state = 42, learning_rate = 0.25)

In [ ]:
#cat_model.fit(train_scaled, train.TARGET)

In [ ]:
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
#xgb_model = XGBClassifier(n_estimators = 500, silent=True, learning_rate = 0.1)

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
#xgb_model.fit(train_scaled, train.TARGET)

In [ ]:
test_y_cat = pd.DataFrame(xgb_model.predict_proba(test_scaled))
submission_cat = pd.concat([test.SK_ID_CURR, test_y_cat], axis=1).drop(0, axis = 1)
submission_cat.columns = ['SK_ID_CURR', 'Target']

In [ ]:
submission_cat.to_csv('xgb_model10.csv', index=False)

In [ ]:
#submission_cat['Target'] = submission_cat['Target'].apply(lambda x: (x - min_sub_cat) / (max_sub_cat - min_sub_cat))

In [ ]:
#cat_submission = pd.read_csv('cat_lr75.csv')
xgb_submission = pd.read_csv('xgb1.csv')

In [ ]:
submission_cat['Target'] = (submission_cat['Target']+ xgb_submission['Target']) /2

In [ ]:
submission_cat.to_csv('combined11.csv', index = False)

In [ ]:
import lightgbm

In [ ]:
train_data = lightgbm.Dataset(train_scaled, train.TARGET)

In [ ]:
parameters = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    'boosting': 'gbdt',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.05,
    'verbose': 0
}

In [ ]:
model = lightgbm.train(parameters,
                       train_data,
#                       valid_sets=test_data,
                       num_boost_round=10000,
#                       early_stopping_rounds=100
                      )

In [ ]:
preds = pd.DataFrame(model.predict(test_scaled))

In [ ]:
submission_lgbm = pd.concat([test.SK_ID_CURR, preds], axis = 1)

In [ ]:
submission_lgbm.columns = ['SK_ID_CURR', 'Target']

In [ ]:
submission_cat.to_csv('lgbm2.csv', index=False)

In [ ]:
submission_cat.head()

In [ ]:
pwd

In [ ]:
combined_10 = pd.read_csv('combined10.csv')

In [ ]:
combined_10['Target'] = (combined_10['Target'] + submission_cat['Target']) /2

In [ ]:
combined_10.to_csv('new_combined.csv', index=False)